In [ ]:
# mount google drive and secure GPU
import os, os.path, datetime
import keras
from google.colab import drive, runtime
import time
drive.mount('/content/drive')

"""some variables"""

data_name = "10k_1000_250_0"
subjects = [30,32,33,34,35,36,37,38,40,42,43,45,46,48,49,52,53,54,55,58,60,61,62,63,66,67,68,69,70,72,73,74,77,79,80,82,84,85,87,88,89,90,91,92,93,94,95,96,98,99,100,101,102,104,105,106,107,108,110,111,112,113,114,115,116,118,119,120,122,123,126,127,128,130,131,132,133,135,138,139,142,143,146,147,150,151,154,156,157,158,159,160,161,162,163]
test_subjects = [1,2,3,4,5,6,7,8,9,10,11,12,13,16,17,18,19,20,21,23,24,25,27,28] # if empty it will not use test subjects
learning_rate = 0.00001 #[] is default of 0.001

window_length = 10000
trials = [1]
model_to_use = 'resnet50_12channels' #'parallel_channels_LSTM, 'parallel_channels_conv1D', 'resnet50', 'resnet50_12channels'
epochs = 1

flag_shuffle_train = 0 # maybe this can go, if we are shuffling subjects beforehand anyways
flag_shuffle_subjects = 0
flag_plot = 0
flag_top_5_accuracy = 0

output_variable = "seconds_10k" #"speed", "subject_id", "seconds_10k"
percentage_of_data_to_be_used = 1 # number between 0 and 100. 100 is all data, everything else takes the percentage


val_split = 0.2
test_split = 0.2

dropout = 0
batch_size = 32

early_stopping_min_delta = 0
early_stopping_patience = 3000
reinitialize_epochs = 1000


kernel_size_convolution = 250
input_model = keras.Input(shape = (100,60,12,1)) #keras.Input(shape = (126,40,12,1))  #keras.Input(shape = (10000,12)) # input for first layer
input_resnet = keras.Input(shape=(126,40,12)) # input after my own layers into the resnet
resnet_trainable = True
weights_to_use = 'seconds_10k_1' #'imagenet', None




# import PredictPower_functions
%cd /content/drive/My\ Drive/Running Plantiga Project/Code
import script_classification as script_classification
#import PredictPower_functions as ppf
#import openloop_algorithms as oa


#check if GPU is available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))


%cd /content/drive/My\ Drive/Running Plantiga Project
dir_root = os.getcwd() + '/'

#This is if you want to load all data into google cloud first

try:
  # create all directories in google cloud storage
  os.mkdir('/content/Running Plantiga Project/')
  os.mkdir('/content/Running Plantiga Project/Code/')
  os.mkdir('/content/Running Plantiga Project/Data/')
  os.mkdir('/content/Running Plantiga Project/Data/Prepared/')
  os.mkdir('/content/Running Plantiga Project/Data/Prepared/tfrecords/')
  #os.mkdir('/content/Running Plantiga Project/Data/Prepared/tfrecords/10k_1000_250_0/')
  os.mkdir('/content/Running Plantiga Project/Data/Results/')
  os.mkdir('/content/Running Plantiga Project/Data/Results/models_info/')
  os.mkdir('/content/Running Plantiga Project/Data/Results/models_trained/')
except:
  print('Paths already exist')


#!cp '/content/drive/MyDrive/Running Plantiga Project/' -R '/content/'

# move all relevant code and data
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Code/script_classification.py' '/content/Running Plantiga Project/Code/'
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Code/functions_classification.py' '/content/Running Plantiga Project/Code/'
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Data/Subject_seconds_scores.csv' '/content/Running Plantiga Project/Data/'
!gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Data/Prepared/tfrecords/10k_1000_250_0/' '/content/Running Plantiga Project/Data/Prepared/tfrecords/'

if weights_to_use != (None and 'imagenet'):
  !gsutil -m cp -r '/content/drive/MyDrive/Running Plantiga Project/Data/Results/models_trained/{weights_to_use}.h5' '/content/Running Plantiga Project/Data/Results/models_trained/'

# unmount google drive
#from google.colab import drive
#drive.flush_and_unmount()

# import PredictPower_functions
%cd /content/Running Plantiga Project/Code
import script_classification as script_classification
#import PredictPower_functions as ppf
#import openloop_algorithms as oa

%cd /content/Running Plantiga Project
dir_root = os.getcwd() + '/'



#check if GPU is available
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))







name_ID = script_classification.script_classification(data_name, subjects, val_split, test_split, flag_shuffle_train, flag_plot, output_variable,
                          epochs, dropout, batch_size, early_stopping_min_delta, early_stopping_patience,
                          dir_root, trials, window_length, kernel_size_convolution, model_to_use,
                          weights_to_use, input_model, input_resnet, resnet_trainable, reinitialize_epochs, flag_shuffle_subjects,
                          flag_top_5_accuracy, test_subjects, learning_rate, percentage_of_data_to_be_used)

# only if using google cloud storage to afterwards load the data over to drive (which usually does not work anyways)

# remount drive and move my result files
drive.mount('/content/drive')
!gsutil -m cp -r '/content/Running Plantiga Project/Data/Results/models_info/{name_ID}.pkl' '/content/drive/MyDrive/Running Plantiga Project/Data/Results/models_info/'
!gsutil -m cp -r '/content/Running Plantiga Project/Data/Results/models_trained/{name_ID}.h5' '/content/drive/MyDrive/Running Plantiga Project/Data/Results/models_trained/'



time.sleep(10)
!kill -9 -1





Mounted at /content/drive
/content/drive/My Drive/Running Plantiga Project/Code
Found GPU at: /device:GPU:0
/content/drive/My Drive/Running Plantiga Project
Copying file:///content/drive/MyDrive/Running Plantiga Project/Code/script_classification.py...
-
Operation completed over 1 objects/11.7 KiB.                                     
Copying file:///content/drive/MyDrive/Running Plantiga Project/Code/functions_classification.py...
/ [1/1 files][ 26.0 KiB/ 26.0 KiB] 100% Done                                    
Operation completed over 1 objects/26.0 KiB.                                     
Copying file:///content/drive/MyDrive/Running Plantiga Project/Data/Subject_seconds_scores.csv...
/ [1/1 files][  4.9 KiB/  4.9 KiB] 100% Done                                    
Operation completed over 1 objects/4.9 KiB.                                      
Copying file:///content/drive/MyDrive/Running Plantiga Project/Data/Prepared/tfrecords/10k_1000_250_0/Treadmill/speed0/SENSOR001.tfrecords..